# Agentive Chatbot

In [31]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
import json
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")

memory_routes = []
memory_dir = "memory"

for filename in os.listdir(memory_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(memory_dir, filename)
        with open(filepath, 'r') as file:
            memory_data = yaml.safe_load(file)
        
        route = Route(
            name=memory_data['route_name'],
            utterances=memory_data['utterances']
            )
        memory_routes.append(route)

encoder = OpenAIEncoder()
memory_rl = RouteLayer(encoder = encoder, routes = memory_routes)

#### Variables to store the default system prompt and the scenario prompt

In [32]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

#### Defines logic for updating, activating, and deactivating prompts

In [33]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []  # Store all conversations
    
    def query(self, query: str, print_response: bool = True) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    if print_response:
                        print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)  # Remove the role message for the next query
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    # If it's a string, write it directly
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    # If it's a list (e.g., self.messages[-2:]), handle it accordingly
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

rpa = ChatBot(api_key=api_key, role = system_prompt)

mem = ChatBot(api_key = api_key, role = "Summarise the user input in a short bullet point")

In [34]:
#returns one route per input
def classify_content(prompt: str) -> str | list[str]:
    r = memory_rl(prompt)
    return r.name

def update_memory(user_input: str) -> None:
    mem.query(user_input, print_response = False)
    summary = mem.messages[-1]["content"]

    # Prepare the text to be appended
    entry = f"{summary}\n"

    # Append the entry to the memory.txt file
    with open("memory.txt", "a") as file:
        file.write(entry)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    rpa.query(pasted_content)
    route = classify_content(pasted_content)
    if route == "dog_memories":
        print("route: dog")
        update_memory(pasted_content)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

# Additional input field for pasted content
pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')  # Adjust height as needed
)
# Button for pasted content submission
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'  # Change to your desired color

# Button for saving chat history
save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'  # Change to your desired color

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

# Arrange buttons side by side and center them using HBox
button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

# Display the input fields and the button layout
display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

Hi there! How's your day going?

I'm really sorry to hear that. Losing a pet is incredibly tough. How are you holding up, and is there anything specific you'd like to talk about or share about your dogs?

route: dog
- User is feeling sad due to the loss of their two dogs.

Bruno and Lola sound like they were really special. Cockapoos and Cavapoos are such loving dogs. Do you have any favorite memories with them that you'd like to share? Sometimes talking about the good times can be comforting.

route: dog
- User's dogs, Bruno (a Cockapoo) and Lola (a Cavapoo), have passed away.

Oh, I see. Are they missing? That must be really stressful. Have you been able to search the area or put up any notices to help find them?

